In [2]:
import math as m
import numpy as np
import matplotlib as mpl
from PIL import Image as im
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import sklearn.datasets as datasets

TEMPFILE_NAME = "temp.png"

def save_to_gif(filename, images, duration=100):
    images[0].save(
        filename,
        optimize=False,
        save_all=True,
        append_images=images[1:],
        loop=0,
        duration=duration,
    )

np.random.seed(1)

COLORS = np.array(['purple', 'green', 'blue'])

# custom CMAP
cvals  = [0, .5, 1]
colors = ['blue', 'white', 'purple']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'purple']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_PURPLE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'green']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_GREEN = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'blue']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_BLUE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

CENTERS = [[0, 0]]
STDEV = 1
DATA, _ = datasets.make_blobs(
        n_samples=200,
        centers=CENTERS,
        cluster_std=STDEV,
        random_state=1
    )
xlim_min = CENTERS[0][0] - 3 * STDEV
xlim_max = CENTERS[0][0] + 3 * STDEV
ylim_min = CENTERS[0][1] - 3 * STDEV
ylim_max = CENTERS[0][1] + 3 * STDEV

def generate_line_data(t, w1, w2, b):
    X = np.array(list(filter(lambda x : w1 * x[0] + w2 * x[1] + b < -.5 or w1 * x[0] + w2 * x[1] + b > .5, t)))
    Y = np.array([0 if w1 * x[0] + w2 * x[1] + b >= 0 else 2 for x in X])
    return X, Y

def sigmoid(x):
    e = np.exp(-x)
    return 1 / (1 + e)

def generate_3_classes_1D(SIZE):
    X = np.linspace(0, 5 * SIZE, SIZE) + np.random.randn(SIZE) * 4
    Y = np.array([0 if x < 18 else 1 if x < 60 else 2 for x in X])
    return X, Y

def generate_3_classes_2D(t, w1, w2):
    f1 = lambda x1, x2: w1[0] * x1 + w1[1] * x2
    f2 = lambda x1, x2: w2[0] * x1 + w2[1] * x2
    X = np.array(list(filter(lambda x : (f1(x[0], x[1]) < -.5 or f1(x[0], x[1]) > .5) and (f2(x[0], x[1]) < -.5 or f2(x[0], x[1]) > .5), t)))
    Y = np.array([0 if f1(x[0], x[1]) >= 0 and f2(x[0], x[1]) >= 0 else 1 if (f1(x[0], x[1]) < 0 and f2(x[0], x[1]) >= 0) else 2 for x in X])
    return X, Y

# OTHER FUNCTIONS
def generate_circle_data(t):
    X = np.array(list(filter(lambda x : (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 < 1 or (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 > 1.5, t)))
    Y = np.array([0 if (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 >= 1 else 2 for x in X])
    return X, Y

def generate_square_data(t):
    X = np.array(list(filter(lambda x : x[0]**2 - x[1] < .4 or x[0]**2 - x[1] > 1.1, t)))
    Y = np.array([1 if x[0]**2 - x[1] >= .75 else 0 for x in X])
    return X, Y

def generate_curve_data(t):
    X = np.array(list(filter(lambda x : m.cos(4*x[0]) - x[1] < -.5 or m.cos(4*x[0]) - x[1] > .5, t)))
    Y = np.array([1 if m.cos(4*x[0]) - x[1] >= 0 else 0 for x in X])
    return X, Y

## 3 classes normalized

In [ ]:
X, Y = generate_3_classes_1D(20) # TO BE USED FOR THE NEXT FEW CELLS
fig, ax = plt.subplots()
ax.scatter(X, np.zeros_like(X), color=COLORS[Y].tolist(), s=100, alpha=.4)
ax.set_ylim(-0.1, 1.1)
fig.savefig('048.png')
plt.close()

In [ ]:
def snap_ind(models, X, Y):

    Y_purple = np.array([1 if y == 0 else 0 for y in Y])
    Y_green = np.array([1 if y == 1 else 0 for y in Y])
    Y_blue = np.array([1 if y == 2 else 0 for y in Y])

    _ = models[0].fit(X, Y_purple, epochs= 1)
    _ = models[1].fit(X, Y_green, epochs= 1)
    _ = models[2].fit(X, Y_blue, epochs= 1)

    xplot = np.linspace(0, 100, 300)
    green = models[1].predict(xplot)
    purple = models[0].predict(xplot)
    blue = models[2].predict(xplot)

    total = purple + blue + green

    fig, ax = plt.subplots(frameon=False)
    ax.scatter(X, np.zeros_like(X), color=COLORS[Y].tolist(), s=100, alpha=.4)
    ax.plot(xplot, purple / total, color=COLORS[0], alpha=.4, linewidth=2)
    ax.plot(xplot, green / total, color=COLORS[1], alpha=.4, linewidth=2)
    ax.plot(xplot, blue / total, color=COLORS[2], alpha=.4, linewidth=2)

    ax.set_ylim(-0.1, 1.1)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

purpleVrest = keras.models.Sequential()
purpleVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
purpleVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

greenVrest = keras.models.Sequential()
greenVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
greenVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

blueVrest = keras.models.Sequential()
blueVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
blueVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

images = []

for _ in range(200):
    images.append(snap_ind([purpleVrest, greenVrest, blueVrest], X, Y))

save_to_gif('049.gif', images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - loss: 27.3410
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 21.0658
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - loss: 20.0043
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 12.8731
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 16.5764
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 15.0663
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3380
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.0878
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 10.1294
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.5615
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 7.6003
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - lo

## not learning from each other

In [ ]:
def snap_ind(models, X, Y):

    Y_purple = np.array([1 if y == 0 else 0 for y in Y])
    Y_blue = np.array([1 if y == 2 else 0 for y in Y])

    _ = models[0].fit(X, Y_purple, epochs= 1)
    _ = models[2].fit(X, Y_blue, epochs= 1)

    xplot = np.linspace(0, 100, 300)
    green = models[1].predict(xplot)
    purple = models[0].predict(xplot)
    blue = models[2].predict(xplot)

    total = purple + blue + green

    fig, ax = plt.subplots(frameon=False)
    ax.scatter(X, np.zeros_like(X), color=COLORS[Y].tolist(), s=100, alpha=.4)
    ax.plot(xplot, purple / total, color=COLORS[0], alpha=.4, linewidth=2)
    ax.plot(xplot, green / total, color=COLORS[1], alpha=.4, linewidth=2)
    ax.plot(xplot, blue / total, color=COLORS[2], alpha=.4, linewidth=2)

    ax.set_ylim(-0.1, 1.1)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

purpleVrest = keras.models.Sequential()
purpleVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
purpleVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

greenVrest = keras.models.Sequential()
greenVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
greenVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

blueVrest = keras.models.Sequential()
blueVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
blueVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

images = []

for _ in range(200):
    images.append(snap_ind([purpleVrest, greenVrest, blueVrest], X, Y))

save_to_gif('050.gif', images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - loss: 2.2235
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - loss: 31.2127
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.7638
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 21.2176
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.3049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 11.2259
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.8468
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.5147
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3934
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/s

## 3 classes (1 - (p + b))

In [ ]:
def snap_ind(models, X, Y):

    Y_purple = np.array([1 if y == 0 else 0 for y in Y])
    Y_blue = np.array([1 if y == 2 else 0 for y in Y])

    _ = models[0].fit(X, Y_purple, epochs= 1)
    _ = models[2].fit(X, Y_blue, epochs= 1)

    xplot = np.linspace(0, 100, 300)
    purple = models[0].predict(xplot)
    blue = models[2].predict(xplot)
    green = 1 - (purple + blue)

    total = purple + blue + green

    fig, ax = plt.subplots(frameon=False)
    ax.scatter(X, np.zeros_like(X), color=COLORS[Y].tolist(), s=100, alpha=.4)
    ax.plot(xplot, purple / total, color=COLORS[0], alpha=.4, linewidth=2)
    ax.plot(xplot, green / total, color=COLORS[1], alpha=.4, linewidth=2)
    ax.plot(xplot, blue / total, color=COLORS[2], alpha=.4, linewidth=2)

    ax.set_ylim(-0.1, 1.1)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

purpleVrest = keras.models.Sequential()
purpleVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
purpleVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

greenVrest = keras.models.Sequential()
greenVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
greenVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

blueVrest = keras.models.Sequential()
blueVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
blueVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

images = []

for _ in range(200):
    images.append(snap_ind([purpleVrest, greenVrest, blueVrest], X, Y))

save_to_gif('051.gif', images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 0.8143
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - loss: 48.0803
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3642
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 38.0841
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.3195
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 28.0887
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2151
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 18.0943
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.3904
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 8.1130
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5572
1/1 ━━━━

## 3 classes (1 - (p + g))

In [ ]:
def snap_ind(models, X, Y):

    Y_purple = np.array([1 if y == 0 else 0 for y in Y])
    Y_green = np.array([1 if y == 1 else 0 for y in Y])

    history = models[0].fit(X, Y_purple, epochs= 1)
    history = models[1].fit(X, Y_green, epochs= 1)

    xplot = np.linspace(0, 100, 300)
    purple = models[0].predict(xplot)
    green = models[1].predict(xplot)
    blue = 1 - (purple + green)

    total = purple + blue + green

    fig, ax = plt.subplots(frameon=False)
    ax.scatter(X, np.zeros_like(X), color=COLORS[Y].tolist(), s=100, alpha=.4)
    ax.plot(xplot, purple / total, color=COLORS[0], alpha=.4, linewidth=2)
    ax.plot(xplot, green / total, color=COLORS[1], alpha=.4, linewidth=2)
    ax.plot(xplot, blue / total, color=COLORS[2], alpha=.4, linewidth=2)

    ax.set_ylim(-0.1, 1.1)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

purpleVrest = keras.models.Sequential()
purpleVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
purpleVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

greenVrest = keras.models.Sequential()
greenVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
greenVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

blueVrest = keras.models.Sequential()
blueVrest.add(keras.layers.Dense(1, input_dim=1, activation="sigmoid"))
blueVrest.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=3e-1))

images = []

for _ in range(200):
    images.append(snap_ind([purpleVrest, greenVrest, blueVrest], X, Y))

save_to_gif('052.gif', images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - loss: 0.5925
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - loss: 0.6336
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2144
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 4.5290
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.2933
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.3225
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2042
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.0341
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3586
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.4321
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.5346
1/1 ━━━━━━

## 3 Classes softmax (only train green and purple)

In [ ]:
def snap(model, X, Y, i):

    a = model.get_weights()

    _ = model.fit(X, keras.utils.to_categorical(Y), epochs= 1)

    b = model.get_weights()

    a[0][0][i] = b[0][0][i] # weight
    a[1][i] = b[1][i] # bias
    model.set_weights(a)

    xplot = np.linspace(0, 100, 300)
    predictions = model.predict(xplot)
    fig, ax = plt.subplots(frameon=False)
    ax.scatter(X, np.zeros_like(X), color=COLORS[Y].tolist(), s=100, alpha=.4)
    for i in range(3):
        ax.plot(xplot, predictions[:, i], color=COLORS[i], alpha=.4, linewidth=2)

    ax.set_ylim(-0.1, 1.1)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=1, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

images = []

for _ in range(200):
    images.append(snap(model, X, Y, 0))
    images.append(snap(model, X, Y, 1))

save_to_gif('053.gif', images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - loss: 6.6460
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 6.4938
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.0033
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.8511
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.3713
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.2193
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.8121
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.6602
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 2.0779
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.9263
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.1237
10/10 ━━━━

## 3 classes only train green

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=1, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

images = []

for _ in range(200):
    images.append(snap(model, X, Y, 1))

save_to_gif('054.gif', images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 22.0581
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 20.5614
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 19.0648
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 17.5683
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 16.0720
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 14.5759
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 13.0799
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 11.5842
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 10.0887
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8.5940
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 7.1013
10

## 3 classes 3D

In [ ]:
class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xplot = np.linspace(-3, 3, 100)
        x, y = np.meshgrid(xplot, xplot)
        meshData = np.c_[x.ravel(), y.ravel()]
        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = plt.axes(projection ='3d')
        ax.view_init(30, -25 - rot)
        ax.scatter(X[:,0], X[:,1], np.ones(Y.shape), color=COLORS[Y].tolist(), s=400, alpha=.4)
        ax.plot_surface(x, y, self.model.predict(meshData)[:, 0].reshape(x.shape), alpha=.2, color=COLORS[0])
        ax.plot_surface(x, y, self.model.predict(meshData)[:, 1].reshape(x.shape), alpha=.2, color=COLORS[1])
        ax.plot_surface(x, y, self.model.predict(meshData)[:, 2].reshape(x.shape), alpha=.2, color=COLORS[2])
        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

X, Y = generate_3_classes_2D(DATA, [-3, 4], [1, 1])

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '055', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1960  
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8813 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6503 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5036 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3940 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3268 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 477u

<Sequential name=sequential_16, built=True>

## Softmax

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

def snap(rot_z, rot_xy):
    xx = np.linspace(-6, 6, 400)
    yy = np.linspace(-6, 6, 400)
    x, y = np.meshgrid(xx, yy)

    z1, z2, z3 = softmax(x, y)

    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(20 + rot_z, -135 - rot_xy)

    ax.plot_surface(x, y, z1, color='purple', alpha=0.3, label='Softmax(x1)')
    ax.plot_surface(x, y, z2, color='g', alpha=0.3, label='Softmax(x2)')
    ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
    ax.contour(x, y, z1, levels=1, colors='purple')
    ax.contour(x, y, z2, levels=1, colors='g')
    ax.contour(x, y, z3, levels=1, colors='b')

    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title('softmax(x_1, x_2)', fontsize=40)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_zlim(-.01, 1.01)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(180):
    images.append(snap(0, i))

save_to_gif("056.gif", images, 10)

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

def snap(i, rot_z, rot_xy):
    xx = np.linspace(-6, 6, 400)
    yy = np.linspace(-6, 6, 400)
    x, y = np.meshgrid(xx, yy)

    z1, z2, z3 = softmax(i * x, y)

    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(20 + rot_z, -135 - rot_xy)

    ax.plot_surface(x, y, z1, color='purple', alpha=0.3, label='Softmax(x1)')
    ax.plot_surface(x, y, z2, color='g', alpha=0.3, label='Softmax(x2)')
    ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
    ax.contour(x, y, z1, levels=1, colors='purple')
    ax.contour(x, y, z2, levels=1, colors='g')
    ax.contour(x, y, z3, levels=1, colors='b')

    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title('softmax(i x_1, x_2)'.replace('i', f'{i:.2f}'), fontsize=40)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_zlim(-.01, 1.01)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(10, 50):
    images.append(snap(i/10, 0, 0))

save_to_gif("057.gif", images, 10)

## Linear function

In [ ]:
X, Y = generate_3_classes_2D(DATA, [1, -1], [1, 1])

def snap(w1, w2, b):
    xx = np.linspace(-6, 6, 400)
    yy = np.linspace(-6, 6, 400)
    x, y = np.meshgrid(xx, yy)

    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(10, -160)

    ax.scatter(X[:,0], X[:,1], w1 * X[:,0] + w2 * X[:,1] + b, color=COLORS[Y].tolist(), s=400, alpha=.4)
    ax.plot_surface(x, y, np.zeros_like(x), color='grey', alpha=.2)
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title(f'{w1:.2f}' + '$x_1 + $' + f'{w2:.2f}' + '$x_2 + $' + f'{b:.2f}', fontsize=40)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_zlim(min(5 * X[:,0] + 5 * X[:,1] + 5), max(5 * X[:,0] + 5 * X[:,1] + 5))
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(10, 50):
    images.append(snap(i/10, 1, 0))

for i in range(10, 50): 
    images.append(snap(5, i/10, 0))

for i in range(10, 50):
    images.append(snap(5, 5, i/10))

save_to_gif("058.gif", images, 10)

## Linear Function perspective

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(-3, 3, 300)
        yy = np.linspace(-3, 3, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(20, -25)

        # ax.plot_surface(x, y, z1, color='purple', alpha=0.3, label='Softmax(x1)')
        # ax.plot_surface(x, y, z2, color='g', alpha=0.3, label='Softmax(x2)')
        ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        # ax.contour(x, y, z1, levels=1, colors='purple')
        # ax.contour(x, y, z2, levels=1, colors='g')
        ax.contour(x, y, z3, levels=1, colors='b')
        
        # ax.scatter(X[:,0], X[:,1], W[0][0] * X[:,0] + W[1][0] * X[:,1] + B[0], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][1] * X[:,0] + W[1][1] * X[:,1] + B[1], color=COLORS[Y].tolist(), s=200, alpha=.4)
        ax.scatter(X[:,0], X[:,1], W[0][2] * X[:,0] + W[1][2] * X[:,1] + B[2], color=COLORS[Y].tolist(), s=200, alpha=.4)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-15, 15)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

X, Y = generate_3_classes_2D(DATA, [-3, 4], [1, 1])

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '059_blue', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0694  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7816 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6061 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4828 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4061 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3418 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3030 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2630
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2332 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2179 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2008 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1947 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1741
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1710 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1634 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1482 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1474 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1389 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.

<Sequential name=sequential_17, built=True>

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(-3, 3, 300)
        yy = np.linspace(-3, 3, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(20, -75)

        # ax.plot_surface(x, y, z2, color='purple', alpha=0.3, label='Softmax(x1)')
        ax.plot_surface(x, y, z1, color='g', alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        # ax.contour(x, y, z2, levels=1, colors='purple')
        ax.contour(x, y, z1, levels=1, colors='g')
        # ax.contour(x, y, z3, levels=1, colors='b')
        
        # ax.scatter(X[:,0], X[:,1], W[0][0] * X[:,0] + W[1][0] * X[:,1] + B[0], color=COLORS[Y].tolist(), s=200, alpha=.4)
        ax.scatter(X[:,0], X[:,1], W[0][1] * X[:,0] + W[1][1] * X[:,1] + B[1], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][2] * X[:,0] + W[1][2] * X[:,1] + B[2], color=COLORS[Y].tolist(), s=200, alpha=.4)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-15, 15)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

X, Y = generate_3_classes_2D(DATA, [-3, 4], [1, 1])

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '059_green', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0785  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8023 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6153 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4885 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3982
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3306 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2949 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2615 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2359 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2142 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2015
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1885 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1704 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1593 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1536 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1507 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1418 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1441 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.

<Sequential name=sequential_18, built=True>

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(-3, 3, 300)
        yy = np.linspace(-3, 3, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(20, -95)

        ax.plot_surface(x, y, z2, color='purple', alpha=0.3, label='Softmax(x1)')
        # ax.plot_surface(x, y, z1, color='g', alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        ax.contour(x, y, z2, levels=1, colors='purple')
        # ax.contour(x, y, z1, levels=1, colors='g')
        # ax.contour(x, y, z3, levels=1, colors='b')
        
        ax.scatter(X[:,0], X[:,1], W[0][0] * X[:,0] + W[1][0] * X[:,1] + B[0], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][1] * X[:,0] + W[1][1] * X[:,1] + B[1], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][2] * X[:,0] + W[1][2] * X[:,1] + B[2], color=COLORS[Y].tolist(), s=200, alpha=.4)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-15, 15)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

X, Y = generate_3_classes_2D(DATA, [-3, 4], [1, 1])

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '059_purple', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9378  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5264 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2619 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9624 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8140 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6681 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5380 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4216 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3745 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3396 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3179 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3005 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2700 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2543 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2391 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2164 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2069 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1845 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 

<Sequential name=sequential_19, built=True>

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(-3, 3, 300)
        yy = np.linspace(-3, 3, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(25, -75)

        # ax.plot_surface(x, y, z2, color='purple', alpha=0.3, label='Softmax(x1)')
        # ax.plot_surface(x_z1, y_z1, z1, color='g', alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        # ax.contour(x, y, z2, levels=1, colors='purple')
        # ax.contour(x, y, z1, levels=1, colors='g')
        # ax.contour(x, y, z3, levels=1, colors='b')
        meshData = np.c_[x.ravel(), y.ravel()]
        ax.contourf(x, y, self.model.predict(meshData)[:, 0].reshape(x.shape), cmap=CMAP_PURPLE, alpha=.4, levels=np.linspace(0.55, 1, 10), zorder=2, offset=-1)
        ax.contourf(x, y, self.model.predict(meshData)[:, 1].reshape(x.shape), cmap=CMAP_GREEN, alpha=.4, levels=np.linspace(0.55, 1, 10), zorder=2, offset=-1)
        ax.contourf(x, y, self.model.predict(meshData)[:, 2].reshape(x.shape), cmap=CMAP_BLUE, alpha=.4, levels=np.linspace(0.55, 1, 10), zorder=2, offset=-1)

        purple = np.where(Y==0)
        X_purple, Y_purple = X[purple], Y[purple]
        green = np.where(Y==1)
        X_green, Y_green = X[green], Y[green]
        blue = np.where(Y==2)
        X_blue, Y_blue = X[blue], Y[blue]
        
        ax.scatter(X_purple[:,0], X_purple[:,1], W[0][0] * X_purple[:,0] + W[1][0] * X_purple[:,1] + B[0], color=COLORS[Y_purple].tolist(), s=200, alpha=.5, zorder=0)
        ax.scatter(X_green[:,0], X_green[:,1], W[0][1] * X_green[:,0] + W[1][1] * X_green[:,1] + B[1], color=COLORS[Y_green].tolist(), s=200, alpha=.5, zorder=0)
        ax.scatter(X_blue[:,0], X_blue[:,1], W[0][2] * X_blue[:,0] + W[1][2] * X_blue[:,1] + B[2], color=COLORS[Y_blue].tolist(), s=200, alpha=.5, zorder=0)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-1, 6)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

X, Y = generate_3_classes_2D(DATA, [-3, 4], [1, 1])

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '059_all', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5722  
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 383us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 379us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 387us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4267 
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 388us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 369us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 355us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3541 
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 390us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 359us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 378us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2945 
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 405us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 343us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 326us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2569
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 360us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 354us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 377us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2183 
2813/28

<Sequential name=sequential_20, built=True>

## Sigmoid with boundary

In [ ]:
def f(x1, x2, C):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

def snap(C, rot_z, rot_xy, rot_ang):
    xplot = np.linspace(-3, 3, 100)
    x, y = np.meshgrid(xplot, xplot)
    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = plt.axes(projection ='3d')
    ax.view_init(20 + rot_z, -135 - rot_ang, rot_xy)
    ax.scatter(X[:,0], X[:,1], sigmoid(f(X[:,0], X[:,1], C)), color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot_surface(x, y, sigmoid(f(x, y, C)), cmap=CMAP, alpha=.4, lw=0)
    ax.plot(xplot, xplot, np.zeros_like(xplot) + .5, linestyle='--', linewidth=3, color='b', label='$3.6x_1 - 3.6x_2 = 0$')
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title(r'$\sigma(C x_1 - C x_2)$'.replace("C", f'{C:.2f}'), fontsize=40)
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_zlim(-.01, 1.01)
    ax.legend(fontsize=15, loc='upper left')
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(45):
    images.append(snap(3.6, 0, 0, i))

for i in range(45):
    images.append(snap(3.6, 0, 0, 45 - i))

for i in range(70):
    images.append(snap(3.6, i, 0, 0))

for i in range(1, 45):
    images.append(snap(3.6, 70, - i, 0))

for _ in range(15):
    images.append(snap(3.6, 70, -45, 0))

save_to_gif("060.gif", images, 10)

## 3 classes 2D training

In [ ]:
class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xplot = np.linspace(-3, 3, 100)
        x, y = np.meshgrid(xplot, xplot)
        meshData = np.c_[x.ravel(), y.ravel()]
        fig, ax = plt.subplots()
        ax.contourf(x, y, self.model.predict(meshData)[:, 0].reshape(x.shape), cmap=CMAP_PURPLE, alpha=.4)
        ax.contourf(x, y, self.model.predict(meshData)[:, 1].reshape(x.shape), cmap=CMAP_GREEN, alpha=.4)
        ax.contourf(x, y, self.model.predict(meshData)[:, 2].reshape(x.shape), cmap=CMAP_BLUE, alpha=.4)
        ax.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

X, Y = generate_3_classes_2D(DATA, [-3, 4], [1, 1])

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '061', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0007  
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7456 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5493 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4286 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 381us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3576 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3089 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/

<Sequential name=sequential_21, built=True>

## Concentric circles

In [ ]:
X, Y = generate_circle_data(DATA)

plt.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=100, alpha=.9)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.savefig("062.png")
plt.close()

## Neural Network

In [ ]:
class LR():

    def __init__(self, model):
        self.model = model

    def _snap_learning(self, X, Y, filename):
        h = .02
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        meshData = np.c_[xx.ravel(), yy.ravel()]

        fig, ax = plt.subplots(frameon=False)
        ax.scatter(X[:, 0], X[:, 1], color=COLORS[Y].tolist(), s=100, alpha=.9)
        Z = self.model.predict(meshData)
        Z = np.array([z[0] for z in Z]).reshape(xx.shape)
        ax.contourf(xx, yy, Z, alpha=.4, cmap=CMAP)
        fig.savefig(filename + '.png')
        plt.close()
        return np.asarray(im.open(filename + '.png'))

    def animate_learning(self, X, Y_0_1, snap_freq=10, filename='learn', duration=1000, **kwargs):
        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for _ in range(int(epochs / snap_freq)):
            self.model.fit(X, Y_0_1, epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning(X, Y, filename)))

        images[0].save(
            filename + '.gif',
            optimize=False,
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

Y_0_1 = Y.copy()
Y_0_1[np.where(Y_0_1 == 0)] = 1
Y_0_1[np.where(Y_0_1 == 2)] = 0

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="tanh"))
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning(X, Y_0_1, 1, '063', 100, epochs=50, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6939  
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 387us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6747 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 409us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6623 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 2s 465us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6451 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 400us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6263 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 419us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5958 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 429us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5617 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 422us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5310 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 387us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4999 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 433us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4517 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 2s 502us/step
2/2 ━━━━━━━━━━━━━━━

<Sequential name=sequential_22, built=True>

In [ ]:
def snap(i):
    point = [-2, 2]
    reflection = [point[0] + i, point[1] - i]
    xplot = np.linspace(-3, 3, 100)
    fig, ax = plt.subplots()
    ax.plot(xplot, xplot, c='b', linestyle='dashed')
    ax.scatter(point[0], point[1], c='b', s=400, alpha=.5)
    ax.scatter(reflection[0], reflection[1], c='purple', s=400, alpha=.5)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(400):
    images.append(snap(i/100))

save_to_gif("reflection.gif", images, 10)

In [ ]:
def snap(i):
    point = [-2, 2]
    reflection_green = [point[0] + i, point[1]]
    reflection_purple = [point[0], point[1] - i]

    purple = np.linspace(-3, 0, 100)
    green = np.linspace(0, 3, 100)
    blue = np.linspace(0, 3, 100)
    fig, ax = plt.subplots()
    ax.plot(purple, np.zeros_like(purple), c='purple', linestyle='dashed')
    ax.plot(np.zeros_like(green), green, c='g', linestyle='dashed')
    ax.plot(blue, -blue, c='b', linestyle='dashed')

    ax.scatter(point[0], point[1], c='b', s=400, alpha=.5)
    ax.scatter(reflection_green[0], reflection_green[1], c='g', s=400, alpha=.5)
    ax.scatter(reflection_purple[0], reflection_purple[1], c='purple', s=400, alpha=.5)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(400):
    images.append(snap(i/100))

save_to_gif("3class_reflection.gif", images, 10)

In [ ]:
def snap(i):
    point = [-2, 2]
    reflection_green = [2 - i, 2 - i]
    reflection_purple = [-2 + i, -2 + i]

    purple = np.linspace(-3, 0, 100)
    green = np.linspace(0, 3, 100)
    blue = np.linspace(0, 3, 100)
    fig, ax = plt.subplots()
    ax.plot(purple, np.zeros_like(purple), c='purple', linestyle='dashed')
    ax.plot(np.zeros_like(green), green, c='g', linestyle='dashed')
    ax.plot(blue, -blue, c='b', linestyle='dashed')

    ax.scatter(point[0], point[1], c='b', s=400, alpha=.5)
    ax.scatter(reflection_green[0], reflection_green[1], c='g', s=400, alpha=.5)
    ax.scatter(reflection_purple[0], reflection_purple[1], c='purple', s=400, alpha=.5)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(400):
    images.append(snap(i/100))

save_to_gif("3class_reflection_2.gif", images, 10)

## Sandbox

In [ ]:
def f(x1, x2, C):
    return C * x1 - C * x2

def logodds(x):
    return np.log(sigmoid(x) / (1 - sigmoid(x)))

X, Y = generate_line_data(DATA, 1, -1, 0)

def snap(C):
    xplot = np.linspace(-3, 3, 100)
    x, y = np.meshgrid(xplot, xplot)
    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = plt.axes(projection ='3d')
    ax.view_init(0, 45)
    ax.scatter(X[:,0], X[:,1], logodds(f(X[:,0], X[:,1], C)), color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot_surface(x, y, logodds(f(x, y, C)), alpha=.2, color='r')
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    if C == 1:
        ax.set_zlabel(r'$\sigma(x_1 - x_2)$', fontsize=20)
    else:
        ax.set_zlabel(r'$\sigma(\frac{1}{C}x_1 - \frac{1}{C}x_2)$'.replace("C", f'{C:.2f}'), fontsize=20)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    # ax.set_zlim(-.01, 1.01)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

# images = []

# r = np.linspace(-10, 10, 10)
# for i in r:
#     if i < 1 and i > -1:
#         continue
#     if i < 0:
#         images.append(snap(1 / (-i)))
#     if i > 0:
#         images.append(snap(i))

# save_to_gif("sandbox.gif", images, 10)

In [ ]:
CENTERS = [[4, 4]]
STDEV = 1
DATA, _ = datasets.make_blobs(
        n_samples=20000,
        centers=CENTERS,
        cluster_std=STDEV,
        random_state=1
    )
xlim_min = CENTERS[0][0] - 3 * STDEV
xlim_max = CENTERS[0][0] + 3 * STDEV
ylim_min = CENTERS[0][1] - 3 * STDEV
ylim_max = CENTERS[0][1] + 3 * STDEV

def coin(heads, tails):
    return np.random.choice([0, 1], p=[heads, tails])

def generate_coin_data(t):
    X = t
    p_heads = abs(x[0])/(abs(x[0]) + abs(x[1]))
    p_tails = 
    Y = np.array([coin(, abs(x[1])/(abs(x[0]) + abs(x[1]))) for x in X])
    return X, Y

xplot = np.linspace(0, 10, 100)
yplot = np.linspace(15, 25, 100)
nplot = np.linspace(-25, -15, 100)
meshData1 = np.c_[xplot.ravel(), xplot.ravel()]
meshData2 = np.c_[xplot.ravel(), yplot.ravel()]
meshData3 = np.c_[xplot.ravel(), nplot.ravel()]

X, Y = generate_coin_data(meshData1)
COLORS = np.array(['blue', 'purple'])
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=10, alpha=.5)
X, Y = generate_coin_data(meshData2)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=10, alpha=.5)
X, Y = generate_coin_data(meshData3)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=10, alpha=.5)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.savefig("sandbox.png")
plt.close()

/var/folders/89/x_333x1920nb7086021gwhlw0000gn/T/ipykernel_20508/3298844477.py:19: RuntimeWarning: invalid value encountered in scalar divide
  Y = np.array(list(filter(lambda x : not np.isnan(x), [coin(abs(x[0])/(abs(x[0]) + abs(x[1])), abs(x[1])/(abs(x[0]) + abs(x[1]))) for x in X])))


ValueError: probabilities contain NaN